In [5]:
import pandas as pd
import numpy as np
import os
import scipy.io

# --- Feature extraction fonksiyonu (19 feature) ---
def extract_features(x):
    N = len(x)
    f = {}
    f['peak'] = np.max(np.abs(x))
    f['rms'] = np.sqrt(np.mean(x**2))
    f['kurtosis'] = np.mean(((x-np.mean(x))/np.std(x))**4)
    f['crest_factor'] = np.max(np.abs(x))/f['rms']
    f['clearance_factor'] = (np.max(np.abs(x))/np.mean(np.sqrt(np.abs(x))))**2
    f['impulse_factor'] = np.max(np.abs(x))/np.mean(np.abs(x))
    f['shape_factor'] = f['rms']/np.mean(np.abs(x))
    f['skewness'] = np.mean(((x-np.mean(x))/np.std(x))**3)
    f['square_mean_root'] = (np.mean(np.sqrt(np.abs(x))))**2
    f['moment_5'] = np.mean(((x-np.mean(x))/np.std(x))**5)
    f['moment_6'] = np.mean(((x-np.mean(x))/np.std(x))**6)
    f['mean'] = np.mean(x)
    f['shape_factor2'] = f['square_mean_root']/np.mean(np.abs(x))
    f['peak_to_peak'] = np.max(x) - np.min(x)
    f['kurtosis_factor'] = f['kurtosis'] / ((np.mean(x**2))**2)
    f['std'] = np.std(x)
    f['smoothness'] = 1 - (1 + (np.std(x))**2)**0.5
    f['uniformity'] = 1 - np.std(x)/np.mean(x) if np.mean(x)!=0 else 0
    f['normal_neg_loglike'] = 1/(np.std(x)*np.sqrt(2*np.pi)) * np.exp(-(x-np.mean(x))**2/(2*np.std(x)**2)).mean()
    return list(f.values())

# --- Ayarlar ---
fs = 10240
start_idx = int(1.5 * fs)
end_idx = int(4.5 * fs)
segment_length = 1024
root_dir = r"C:\Users\VICTUS\OneDrive\Masaüstü\EECSL"   # Ana .mat dosyalarının olduğu klasör

# Tüm segment log dosyalarının yolunu ver (split ve label ile)
log_files = [
    ('healthy', 'train', r"C:\Users\VICTUS\Pictures\(1.5-4.5s)\Reassigned(1.5-4.5s)\healthy_train_segments.csv"),
    ('healthy', 'test', r"C:\Users\VICTUS\Pictures\(1.5-4.5s)\Reassigned(1.5-4.5s)\healthy_test_segments.csv"),
    ('imbalance', 'train', r"C:\Users\VICTUS\Pictures\(1.5-4.5s)\Reassigned(1.5-4.5s)\imbalance_train_segments.csv"),
    ('imbalance', 'test', r"C:\Users\VICTUS\Pictures\(1.5-4.5s)\Reassigned(1.5-4.5s)\imbalance_test_segments.csv"),
    ('loose', 'train', r"C:\Users\VICTUS\Pictures\(1.5-4.5s)\Reassigned(1.5-4.5s)\loose_train_segments.csv"),
    ('loose', 'test', r"C:\Users\VICTUS\Pictures\(1.5-4.5s)\Reassigned(1.5-4.5s)\loose_test_segments.csv"),
    ('misalignment', 'train', r"C:\Users\VICTUS\Pictures\(1.5-4.5s)\Reassigned(1.5-4.5s)\misalignment_train_segments.csv"),
    ('misalignment', 'test', r"C:\Users\VICTUS\Pictures\(1.5-4.5s)\Reassigned(1.5-4.5s)\misalignment_test_segments.csv"),
]

features_list = [] # Tüm segmentlerden çıkan feature vektörleri burada toplanacak.
meta_list = [] # Her feature’ın split, label, matfile, channel, segment index bilgisi.
mat_cache = {} # Aynı .mat dosyasını tekrar tekrar okumamak için “cache” (performans için).

for label, split, log_path in log_files:
    log_df = pd.read_csv(log_path) # Her bir label ve split kombinasyonu için ilgili segmentlerin bilgisini okur.
    for idx, row in log_df.iterrows():
        # Her segmentin, hangi .mat dosyasına, hangi channela (cur1, cur2), ve hangi segment başlangıç indexine ait olduğunu almak.
        matfile = row['matfile']
        channel = row['channel']
        seg_idx = int(row['start_index'])

        # .mat dosyasını bul (root_dir/label_start/...)
        mat_path = os.path.join(root_dir, f"{label}_start", matfile + ".mat")
        if not os.path.exists(mat_path):
            print(f"Mat dosyası bulunamadı: {mat_path}")
            continue

        # Aynı .mat dosyasını ve kanalı birden fazla okumamak için cache (önbellek) kullanılır.
        cache_key = f"{mat_path}_{channel}"
        if cache_key in mat_cache:
            signal = mat_cache[cache_key]
        else:
            matdata = scipy.io.loadmat(mat_path)
            if channel not in matdata:
                print(f"Kanallar arasında yok: {channel} in {mat_path}")
                continue
            signal = matdata[channel].squeeze()
            signal = signal[start_idx:end_idx]
            mat_cache[cache_key] = signal

        # Segmentin bitiş noktası, sinyal uzunluğunu aşıyorsa segmenti atlar.
        if seg_idx+segment_length > len(signal):
            print(f"Segment sınırı aşıldı: {matfile}, {channel}, {seg_idx}")
            continue

        segment = signal[seg_idx:seg_idx+segment_length] # Belirlenen başlangıç noktasından sonuna kadar veri çeker.
        feat = extract_features(segment) # Bu segment için istatistiksel öznitelik çıkar.
        # Sonuç vektörünü ve segmentin meta bilgisini listeye ekle.
        features_list.append(feat)
        meta_list.append([split, label, matfile, channel, seg_idx])

# CSV dosyası oluşturma işlemi.

# Python listesindeki tüm feature vektörlerini bir numpy array’e çevirir.
features_arr = np.array(features_list) 
# # Her segmentin meta bilgisini (train/test, arıza tipi, dosya adı, kanal adı, segmentin başladığı index) içeren DataFrame oluşturmak.
meta_df = pd.DataFrame(meta_list, columns=['split','label','matfile','channel','start_index']) 
# Her feature sütununa bir isim verip (feat_1, feat_2, ..., feat_19) DataFrame oluşturur.
features_df = pd.DataFrame(features_arr, columns=[f'feat_{i+1}' for i in range(features_arr.shape[1])])
# Meta bilgileri ve feature DataFrame’ini yan yana birleştirerek tek bir tablo (out_df) oluşturur.
out_df = pd.concat([meta_df, features_df], axis=1)
save_dir = r"C:\Users\VICTUS\OneDrive\Masaüstü\EECSL"
out_df.to_csv(os.path.join(save_dir, "all_segments_1d_features.csv"), index=False)
print("Tüm segmentler için feature extraction tamamlandı!")


Tüm segmentler için feature extraction tamamlandı!


In [7]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm import tqdm

# 1D feature csv'yi yükle
df = pd.read_csv("all_segments_1d_features.csv")

# Tüm label'lar ve split'ler için (örnek: train/healthy)
img_root = r"C:\Users\VICTUS\Pictures\(1.5-4.5s)\Reassigned(1.5-4.5s)"
splits = ['train', 'test']
labels = ['healthy', 'imbalance', 'loose', 'misalignment']

for split in splits:
    X1 = [] # 1D featureler.
    X2 = [] # 2D görseller
    y = [] # Etiketler

    for label in labels:
        # O split ve label için .csv satırlarını seç
        # Belirli bir split ve label için tüm segmentlerin DataFrame’i çekilir.
        subset = df[(df['split']==split) & (df['label']==label)].reset_index(drop=True)

        # Görsel klasörünü tara
        img_dir = os.path.join(img_root, split, label)
        img_files = sorted(os.listdir(img_dir), key=lambda x: int(x.split('_')[1].split('.')[0]))
        assert len(img_files) == len(subset), f"{label}/{split} eşleşme sorunu: {len(img_files)} görsel, {len(subset)} feature"

        # Her segment İçin 1D feature, 2D görsel ve label Ekle.
        for i, img_name in enumerate(tqdm(img_files, desc=f"{split}/{label}")):
            # 1D feature
            features = subset.iloc[i, 5:].values.astype(np.float32)   # ilk 5 sütun meta, sonrası feature
            X1.append(features)
            # 2D image
            img_path = os.path.join(img_dir, img_name)
            img = Image.open(img_path).convert('RGB')
            img_arr = np.array(img).astype(np.float32) / 255.0
            X2.append(img_arr)
            # Label
            y.append(labels.index(label))  # label'ı int'e çevir

    # Dataları numpy olarak kaydet (veya modeline doğrudan ver)
    X1 = np.stack(X1)
    X2 = np.stack(X2)
    y = np.array(y)

    np.save(f"X1_{split}.npy", X1)
    np.save(f"X2_{split}.npy", X2)
    np.save(f"y_{split}.npy", y)
    print(f"{split} split için shape: X1={X1.shape}, X2={X2.shape}, y={y.shape}")


train/misalignment: 100%|██████████| 377/377 [00:02<00:00, 181.87it/s]


train split için shape: X1=(1508, 19), X2=(1508, 224, 224, 3), y=(1508,)


test/misalignment: 100%|██████████| 95/95 [00:00<00:00, 151.95it/s]


test split için shape: X1=(380, 19), X2=(380, 224, 224, 3), y=(380,)


In [5]:
import os
import numpy as np
import pandas as pd
import scipy.io
import glob

# ========== Parametreler ==========
fs = 10240
start_idx = int(1.5 * fs)
end_idx   = int(4.5 * fs)
segment_length = 1024

# Yol ayarları (gerekirse değiştir)
root_dir = r"C:\Users\VICTUS\OneDrive\Masaüstü\EECSL"   # .mat klasörlerinin ana dizini
index_map_path = os.path.join(root_dir, "reassigned_index_map.csv")  # 2D generator çıktısı
save_csv_path  = os.path.join(root_dir, "all_segments_1d_features_new.csv")

# ========== Yardımcılar ==========
EPS = 1e-12

def safe_std(x):
    s = np.std(x)
    return s if s > EPS else EPS

def safe_mean_abs(x):
    m = np.mean(np.abs(x))
    return m if m > EPS else EPS

def safe_smr(x):
    # square mean root = (mean(sqrt(|x|)))**2
    smr = (np.mean(np.sqrt(np.abs(x))))**2
    return smr if smr > EPS else EPS

def extract_features(x):
    """
    19 özellik — NaN/Inf korumalı.
    x: 1D numpy array (float)
    """
    x = np.asarray(x, dtype=np.float64)
    N = len(x)
    mu = np.mean(x)
    sd = safe_std(x)
    abs_mean = safe_mean_abs(x)
    smr = safe_smr(x)
    rms = np.sqrt(np.mean(x**2) + EPS)
    maxabs = np.max(np.abs(x))
    z = (x - mu) / sd

    f = {}
    f['peak']               = maxabs
    f['rms']                = rms
    f['kurtosis']           = np.mean(z**4)
    f['crest_factor']       = maxabs / rms
    f['clearance_factor']   = (maxabs / np.mean(np.sqrt(np.abs(x)) + EPS))**2
    f['impulse_factor']     = maxabs / abs_mean
    f['shape_factor']       = rms / abs_mean
    f['skewness']           = np.mean(z**3)
    f['square_mean_root']   = smr
    f['moment_5']           = np.mean(z**5)
    f['moment_6']           = np.mean(z**6)
    f['mean']               = mu
    f['shape_factor2']      = smr / abs_mean
    f['peak_to_peak']       = np.max(x) - np.min(x)
    f['kurtosis_factor']    = f['kurtosis'] / ((np.mean(x**2) + EPS)**2)
    f['std']                = sd
    f['smoothness']         = 1 - np.sqrt(1 + sd**2)
    f['uniformity']         = 1 - sd / (mu if abs(mu) > EPS else (np.sign(mu)*EPS or EPS))
    # normal_neg_loglike: normal PDF ortalaması (adın orijinaline sadık kalındı)
    f['normal_neg_loglike'] = (1.0 / (sd*np.sqrt(2*np.pi))) * np.exp(-((x-mu)**2) / (2*sd**2))
    f['normal_neg_loglike'] = float(np.mean(f['normal_neg_loglike']))
    return [f[k] for k in f]

def resolve_mat_path(label, basename_safe):
    """
    2D index_map'ten gelen 'basename' (muhtemelen boşluklar '_' yapılmış) ile .mat yolunu bulur.
    Deneme sırası:
      1) exact: {label}_start/{basename_safe}.mat
      2) boşluk <-> '_' dönüşümü
      3) wildcard (kısmi eşleşme) — başında/sonunda ek karakter varsa da bulmaya çalış.
    Hiçbiri yoksa None döner.
    """
    base_dir = os.path.join(root_dir, f"{label}_start")

    # 1) exact
    cand = os.path.join(base_dir, basename_safe + ".mat")
    if os.path.exists(cand):
        return cand

    # 2) '_' <-> ' ' dönüşümü
    flipped = basename_safe.replace("_", " ")
    cand2 = os.path.join(base_dir, flipped + ".mat")
    if os.path.exists(cand2):
        return cand2

    # 3) wildcard — her iki varyasyon için dene
    # örn: SOME_NAME -> *SOME*NAME*.mat ve *SOME NAME*.mat
    patt1 = os.path.join(base_dir, f"*{basename_safe}*.mat")
    patt2 = os.path.join(base_dir, f"*{flipped}*.mat")

    hits = glob.glob(patt1)
    if not hits:
        hits = glob.glob(patt2)

    if hits:
        # Eğer birden fazla eşleşme varsa en kısa isimliyi seç (daha muhtemel doğru)
        hits.sort(key=lambda p: len(os.path.basename(p)))
        return hits[0]

    return None

# ========== Ana akış ==========
# 2D tarafın index map'ini oku
idx_df = pd.read_csv(index_map_path)

required_cols = {'split','label','filename','basename','signal_key','seg_start'}
missing = required_cols - set(idx_df.columns)
if missing:
    raise ValueError(f"Index map içinde beklenen kolonlar yok: {missing}")

# Çıktılar
features_list = []
meta_list     = []

# .mat okuma cache (dosya × kanal)
mat_cache = {}

# Satır satır gez: her satır 2D görüntünün birebir tanımıdır
for i, row in idx_df.iterrows():
    split      = str(row['split'])
    label      = str(row['label'])
    filename   = str(row['filename'])       # 2D görsel dosya adı (eşleştirme için saklayacağız)
    basename   = str(row['basename'])       # 2D yazarken "safe" olabilir (alt çizgi)
    channel    = str(row['signal_key'])     # 'cur1' veya 'cur2'
    seg_start  = int(row['seg_start'])      # 1.5–4.5s aralığındaki segment başlangıcı (örnek cinsinden)

    # İlgili .mat dosyasının yolunu çöz
    mat_path = resolve_mat_path(label, basename)
    if mat_path is None:
        print(f"[UYARI] .mat bulunamadı -> label={label}, basename={basename}")
        continue

    # Cache anahtarı ve okuma
    cache_key = f"{mat_path}__{channel}"
    if cache_key in mat_cache:
        signal = mat_cache[cache_key]
    else:
        try:
            matdata = scipy.io.loadmat(mat_path)
        except Exception as e:
            print(f"[UYARI] .mat okunamadı: {mat_path} ({e})")
            continue

        if channel not in matdata:
            print(f"[UYARI] Kanal yok: {channel} in {mat_path}")
            continue

        signal_full = np.asarray(matdata[channel]).squeeze().astype(np.float64)
        # 2D taraftaki ile aynı ön-işleme: normalize + 1.5–4.5s kırpma
        signal_full = (signal_full - np.mean(signal_full)) / (np.std(signal_full) + EPS)
        signal = signal_full[start_idx:end_idx]
        mat_cache[cache_key] = signal

    # segment sınır kontrolü
    if seg_start + segment_length > len(signal):
        print(f"[UYARI] Segment sınırı aşıldı: {os.path.basename(mat_path)}, {channel}, start={seg_start}")
        continue

    segment = signal[seg_start:seg_start+segment_length]

    # Özellik çıkar
    feat_vec = extract_features(segment)
    features_list.append(feat_vec)

    # Meta: split,label,filename (2D ile birebir eşleşme!), matfile stem, channel, start_index
    mat_stem = os.path.splitext(os.path.basename(mat_path))[0]
    meta_list.append([split, label, filename, mat_stem, channel, seg_start])

# NumPy/DF ve kaydet
if not features_list:
    raise RuntimeError("Hiç özellik çıkarılamadı. Lütfen index_map ve .mat yollarını kontrol et.")

features_arr = np.array(features_list, dtype=np.float64)
features_df  = pd.DataFrame(features_arr, columns=[f"feat_{i+1}" for i in range(features_arr.shape[1])])

meta_df = pd.DataFrame(meta_list, columns=['split','label','filename','matfile','channel','start_index'])

# Çıktı CSV: (split, label, filename, matfile, channel, start_index, feat_1..feat_19)
out_df = pd.concat([meta_df, features_df], axis=1)
out_df.to_csv(save_csv_path, index=False, encoding="utf-8-sig")

print(f"[OK] 1D feature extraction tamamlandı.")
print(f"Çıktı: {save_csv_path}")
print(f"Toplam segment: {len(out_df)}")


[OK] 1D feature extraction tamamlandı.
Çıktı: C:\Users\VICTUS\OneDrive\Masaüstü\EECSL\all_segments_1d_features_new.csv
Toplam segment: 1888
